In [3]:
import pandas as pd 
import matplotlib
import numpy as np
import scipy as sp #collection of functions for scientific computing and advance mathematics
import IPython
from IPython import display
import sklearn
import random
import time

In [4]:
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics


import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns


pd.options.display.max_rows = 4000

mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

In [5]:
data_raw = pd.read_csv('train.csv')
data_val   = pd.read_csv('test.csv')
data1 = data_raw.copy(deep = True)
data_cleaner = [data1, data_val]

print (data_raw.info())
data_raw.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [6]:
print('Train columns with null values:\n', data1.isnull().sum().sort_values(ascending=False))
print("-"*10)

print('Test/Validation columns with null values:\n', data_val.isnull().sum().sort_values(ascending=False))
print("-"*10)



Train columns with null values:
 PoolQC           1453
MiscFeature      1406
Alley            1369
Fence            1179
MasVnrType        872
FireplaceQu       690
LotFrontage       259
GarageQual         81
GarageFinish       81
GarageType         81
GarageYrBlt        81
GarageCond         81
BsmtFinType2       38
BsmtExposure       38
BsmtCond           37
BsmtQual           37
BsmtFinType1       37
MasVnrArea          8
Electrical          1
Condition2          0
BldgType            0
Neighborhood        0
LandSlope           0
LotConfig           0
Condition1          0
LandContour         0
LotShape            0
Street              0
LotArea             0
MSSubClass          0
MSZoning            0
Id                  0
Utilities           0
HouseStyle          0
Foundation          0
ExterQual           0
ExterCond           0
BsmtUnfSF           0
TotalBsmtSF         0
Heating             0
BsmtFinSF1          0
Exterior2nd         0
Exterior1st         0
RoofMatl            0

In [40]:
most_nulls = data_val.isnull().sum().sort_values(ascending=False)
print(most_nulls)
num_of_unique_in_most_nulls = pd.DataFrame(columns=['Column','Number of unique'])
for column_name, null_count in most_nulls.items():
    new_row = pd.DataFrame({'Column': [column_name], 'Number of unique': [data_val[column_name].fillna('NaN').nunique()]})
    num_of_unique_in_most_nulls = pd.concat([num_of_unique_in_most_nulls, new_row], ignore_index=True)
   

num_of_unique_in_most_nulls = num_of_unique_in_most_nulls.sort_values(ascending=False, by='Number of unique')

num_of_unique_in_most_nulls





PoolQC           1456
MiscFeature      1408
Alley            1352
Fence            1169
MasVnrType        894
FireplaceQu       730
LotFrontage       227
GarageYrBlt        78
GarageCond         78
GarageFinish       78
GarageQual         78
GarageType         76
BsmtCond           45
BsmtQual           44
BsmtExposure       44
BsmtFinType1       42
BsmtFinType2       42
MasVnrArea         15
MSZoning            4
BsmtHalfBath        2
Utilities           2
Functional          2
BsmtFullBath        2
BsmtFinSF1          1
Exterior1st         1
TotalBsmtSF         1
BsmtUnfSF           1
BsmtFinSF2          1
SaleType            1
KitchenQual         1
GarageCars          1
GarageArea          1
Exterior2nd         1
Condition2          0
Condition1          0
Neighborhood        0
BldgType            0
LotConfig           0
LandContour         0
LotShape            0
MSSubClass          0
Street              0
LotArea             0
Id                  0
LandSlope           0
ExterCond 

,Column,Number of unique
43,Id,1459
42,LotArea,1106
48,GrLivArea,879
26,BsmtUnfSF,794
51,1stFlrSF,789
25,TotalBsmtSF,737
23,BsmtFinSF1,670
31,GarageArea,460
50,2ndFlrSF,407
17,MasVnrArea,304
